In [ ]:
!pip install rdkit-pypi
!pip install mordred

In [ ]:
from rdkit import Chem
from mordred import Calculator, descriptors
import pandas as pd
import numpy as np
 


In [ ]:
train_data=pd.read_csv("Untitled Folder/train_dataframe.csv")
train_data[train_data['acc_SMILES']=="bad mol"]

In [ ]:
train_data

,Reported Donor,Reported Acceptor,acc_SMILES,don_SMILES,PCE (%)
0,PPDT2FBT,BTDT2R,CCOc1c(OCC)c(-c2ccc(/C=C3\SC(=S)N(CC)C3=O)s2)c...,CCOc1ccc(OCC)c(-c2ccc(-c3c(F)c(F)c(-c4ccc(-c5c...,6.90
1,PTB7-Th,BTDT2R,CCOc1c(OCC)c(-c2ccc(/C=C3\SC(=S)N(CC)C3=O)s2)c...,CCOC(=O)c1sc2c(-c3cc4c(-c5ccc(CC)s5)c5sc(-c6sc...,8.19
2,P3HT,BTDT2R,CCOc1c(OCC)c(-c2ccc(/C=C3\SC(=S)N(CC)C3=O)s2)c...,CCc1cc(C)sc1-c1sc(-c2cc(CC)c(-c3sc(-c4cc(CC)c(...,5.09
3,PTB7-Th,CDTBM,CCC1(CC)c2cc(-c3ccc(C=C(C#N)C#N)c4nsnc34)sc2-c...,CCOC(=O)c1sc2c(-c3cc4c(-c5ccc(CC)s5)c5sc(-c6sc...,5.60
4,PTB7-Th,IDTT-2BM,CCc1ccc(C2(c3ccc(CC)cc3)c3cc4c(cc3-c3sc5cc(-c6...,CCOC(=O)c1sc2c(-c3cc4c(-c5ccc(CC)s5)c5sc(-c6sc...,8.40
...,...,...,...,...,...
1276,PDBT-T1,NaN,O=C(C(C=CC=C1)=C1C/2=C(C#N)/C#N)C2=C/C(S3)=CC4...,CCc1ccc(-c2c3sc4cc(-c5ccc(-c6sc(-c7ccc(-c8cc9s...,7.50
1277,P3HT,NaN,O=C1/C(C2=CC=C(C(S3)=CC4=C3C(SC(C(C=C5)=CC6=C5...,CCc1cc(C)sc1-c1sc(-c2cc(CC)c(-c3sc(-c4cc(CC)c(...,0.23
1278,P3HT,NaN,O=C1/C(C2=CC=C(C3=C(C=CC=C4)C4=C(C5=CC=CC=C53)...,CCc1cc(C)sc1-c1sc(-c2cc(CC)c(-c3sc(-c4cc(CC)c(...,0.28
1279,P3HT,NaN,O=C1/C(C2=CC=C(C3=CC(C4=CC(N(CC(CC)CCCC)C(/C5=...,CCc1cc(C)sc1-c1sc(-c2cc(CC)c(-c3sc(-c4cc(CC)c(...,0.54


In [ ]:

def calculate_descriptors(smiles):
  """
  Calculates molecular descriptors from a SMILES string.

  Args:
      smiles (str): The input SMILES string.

  Returns:
      dict: A dictionary containing molecular descriptors as keys and their corresponding values.
          Returns None if the SMILES string cannot be converted to an rdkit.Chem.Mol instance.

  Raises:
      None.

  Example:
      >>> calculate_descriptors('CCO')
      {'MaxEStateIndex': 6.01875, 'MinEStateIndex': 0.78125, '...}

  """
  try:
    mol = Chem.MolFromSmiles(smiles)
    if mol is None : 
      return {}
    calc = Calculator(descriptors)
    desc = calc(mol)
    return desc.asdict()
  except ValueError:
        print("Unable to convert SMILES string to rdkit.Chem.Mol instance.")
        return {}

**Generate descriptors of Acceptors smiles**


In [ ]:
uni=pd.DataFrame(np.unique(train_data["acc_SMILES"]))
uni=uni.rename({0 : "acc_SMILES"}, axis='columns')
mask = uni['acc_SMILES'] == 'bad mol'
uni = uni[~mask]
len(uni)

In [ ]:
desc_dfA = pd.DataFrame(list(uni['acc_SMILES'].apply(calculate_descriptors))) 

In [ ]:

new_columns = [col_name + '_acceptor' for col_name in desc_dfA.columns]
desc_dfA.columns = new_columns
desc_dfA

In [ ]:
train_data_with_acc_smiles=pd.concat([uni, desc_dfA], axis=1)

In [ ]:

train_data_with_acc_smiles=pd.merge(train_data_with_acc_smiles, train_data, on='acc_SMILES')

**Generate descriptors of Donors smiles**

In [ ]:
uni=pd.DataFrame(np.unique(train_data["don_SMILES"]))
uni=uni.rename({0 : "don_SMILES"}, axis='columns')
mask = uni['don_SMILES'] == 'bad mol'
uni = uni[~mask]
len(uni)

210

In [ ]:
desc_df = pd.DataFrame(list(uni['don_SMILES'].apply(calculate_descriptors))) 

In [ ]:
new_columns = [col_name + '_donor' for col_name in desc_df.columns]
desc_df.columns = new_columns
desc_df

In [ ]:
train_data_with_don_smiles=pd.concat([uni, desc_df], axis=1)

In [ ]:
train_data_with_smiles=pd.merge(train_data_with_don_smiles, train_data_with_acc_smiles, on='don_SMILES')

In [ ]:
train_data_with_smiles.to_csv('/content/Untitled Folder/train_with_descriptor.csv', index=False)

In [ ]:
train_data_with_smiles

,don_SMILES,ABC_donor,ABCGG_donor,nAcid_donor,nBase_donor,SpAbs_A_donor,SpMax_A_donor,SpDiam_A_donor,SpAD_A_donor,SpMAD_A_donor,...,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,Reported Donor,Reported Acceptor,PCE (%)
0,CCCc1sc(-c2c3cc(-c4cc(C(=O)CC)c(-c5ccc(-c6sc(-...,124.327252,84.118682,0,0,204.028157,2.635164,5.121447,204.028157,1.324858,...,9.233652,43758.0,195.0,540.0,690.0,27.847222,20.000000,PBDT-3TCO,IT-4F,11.77
1,CCN1C(=O)/C(=C\c2csc(-c3ccc(-c4ccc(-c5sc(-c6cc...,68.613631,46.808443,0,0,113.422146,2.538887,4.841794,113.422146,1.350264,...,8.712186,38532.0,185.0,516.0,660.0,24.402778,19.277778,PTIBT,ITIC,5.72
2,CCOC(=O)/C(C#N)=C/c1ccc(-c2sc(-c3cc4c(-c5ccc(C...,96.424552,65.129270,0,0,160.252377,2.651864,5.137436,160.252377,1.335436,...,11.011435,23112.0,146.0,412.0,532.0,22.138889,14.944444,3BDT-5,Y6,10.40
3,CCOC(=O)/C(C#N)=C\c1cc(CC)c(-c2ccc(-c3sc(-c4cc...,66.015914,47.400457,0,0,109.669402,2.632854,5.119109,109.669402,1.305588,...,8.545786,14389.0,123.0,336.0,432.0,18.569444,13.000000,SM-Cl,IDIC,7.73
4,CCOC(=O)/C(C#N)=C\c1cc(CC)c(-c2ccc(-c3sc(-c4cc...,66.015914,47.400457,0,0,109.669402,2.632854,5.119109,109.669402,1.305588,...,7.303266,51681.0,187.0,536.0,664.0,27.458333,22.166667,SM-Cl,NaN,7.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,COC(=O)c1csc(-c2cc(C(=O)OC)c(-c3cc(Cl)c(-c4ccc...,69.269970,49.813381,0,0,114.399305,2.541261,4.812991,114.399305,1.314935,...,8.545786,14389.0,123.0,336.0,432.0,18.569444,13.000000,PDCBT-Cl,IDIC,9.44
1276,COC(=O)c1csc(-c2cc(C(=O)OC)c(-c3cc(Cl)c(-c4ccc...,69.269970,49.813381,0,0,114.399305,2.541261,4.812991,114.399305,1.314935,...,11.011435,23112.0,146.0,412.0,532.0,22.138889,14.944444,PDCBT-Cl,Y6,0.48
1277,COC(=O)c1csc(-c2cc(C(=O)OC)c(-c3cc(Cl)c(-c4ccc...,69.269970,49.813381,0,0,114.399305,2.541261,4.812991,114.399305,1.314935,...,9.233652,43758.0,195.0,540.0,690.0,27.847222,20.000000,PDCBT-Cl,IT-4F,9.05
1278,COC(=O)c1csc(-c2cc(C(=O)OC)c(-c3cc(Cl)c(-c4ccc...,69.269970,49.813381,0,0,114.399305,2.541261,4.812991,114.399305,1.314935,...,8.712186,38532.0,185.0,516.0,660.0,24.402778,19.277778,PDCBT-Cl,ITIC,7.25
